<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/SingleCharacteristic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

In [3]:
#Split between training data, validation data, test data
splitSize = (0.0, 0.765, 0.9, 1.0)

Testing precision on red channel only

In [4]:
array = np.vstack(tfds.as_numpy(dataset[0]))

ranges = list([int(array.shape[0] * x) for x in splitSize])
def getDataset(ranges, idx):
  return ( 
      np.array(list(map(lambda x: x[0][:, :, 0], array[ranges[idx] : ranges[idx+1]]))),
      np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx] : ranges[idx+1]])))
      )

train_images, train_labels = getDataset(ranges, 0)
validation_images, validation_labels = getDataset(ranges, 1)
test_images, test_labels = getDataset(ranges, 2)

<ipython-input-4-39bb5a9fcd49>:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  array = np.vstack(tfds.as_numpy(dataset[0]))
/usr/local/lib/python3.8/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


In [12]:
model = models.Sequential([
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ])

inputs = tf.keras.Input(shape=(300, 150, 1))
layer = layers.Rescaling(1./255.)(inputs)
layer = layers.Conv2D(filters=32, kernel_size=3, activation="sigmoid")(layer)
layer = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(layer)
layer = layers.MaxPooling2D(pool_size=2)(layer)
layer = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(layer)
layer = layers.Flatten()(layer)
outputs = layers.Dense(4, activation="relu")(layer)
layer = layers.Dense(units=64, activation="relu")(layer)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 300, 150, 1)]     0         
                                                                 
 rescaling_1 (Rescaling)     (None, 300, 150, 1)       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 298, 148, 32)      320       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 149, 74, 32)      0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 147, 72, 64)       18496     
                                                                 
 flatten_1 (Flatten)         (None, 677376)            0         
                                                           

In [13]:
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=5)

Epoch 1/5
180/180 [==============================] - 16s 79ms/step - loss: 2.6923 - accuracy: 0.8323 - val_loss: 2.7713 - val_accuracy: 0.8281
Epoch 2/5
180/180 [==============================] - 14s 76ms/step - loss: 2.7037 - accuracy: 0.8323 - val_loss: 2.7713 - val_accuracy: 0.8281
Epoch 3/5
 54/180 [========>.....................] - ETA: 9s - loss: 2.8542 - accuracy: 0.8229

KeyboardInterrupt: ignored

In [8]:
model.evaluate(test_images, test_labels)

24/24 [==============================] - 1s 26ms/step - loss: 2.8191 - accuracy: 0.8251


[2.8190526962280273, 0.8251001238822937]